# Добавление документа в ClickHouse
Этот ноутбук предназначен для добавления документа в базу данных ClickHouse. Документ предварительно обрабатывается, разбивается на субчанки, токенизируется и преобразуется в векторное представление, которое затем сохраняется в базу данных для последующего использования в поиске и анализе данных.


### Установка необходимых библиотек
Для запуска этого блокнота убедитесь, что у вас установлены библиотеки `transformers`, `torch`, `clickhouse_connect` и `accelerate`.

In [1]:
%pip install transformers torch clickhouse_connect accelerate

Note: you may need to restart the kernel to use updated packages.


In [2]:
from __future__ import annotations
import logging
import re
from dataclasses import dataclass
from typing import (
    Any,
    Dict,
    Literal,
    Callable,
    Iterable,
    List,
    Optional,
    Sequence,
    cast,
    Union
)

import clickhouse_connect
import copy
import torch
from transformers import AutoModel, AutoTokenizer

from pydantic import Field, BaseModel
import time

logger = logging.getLogger(__name__)

### Функции для обработки текста и моделей

В данной ячейке определены следующие функции:

1. `mean_pooling(model_output: tuple, attention_mask: torch.Tensor) -> torch.Tensor`: Функция выполняет усреднение токенов входной последовательности на основе attention mask.

2. `txt2embeddings(text: Union[str, List[str]], tokenizer, model, device: str = "cpu") -> torch.Tensor`: Преобразует текст в его векторное представление с использованием модели transformer.

3. `load_models(model: str, device: str = "cpu", torch_dtype: str = "auto") -> tuple`: Загружает токенизатор и модель для указанной предобученной модели.


In [3]:
def mean_pooling(model_output: tuple, attention_mask: torch.Tensor) -> torch.Tensor:
    """
    Выполняет усреднение токенов входной последовательности на основе attention mask.

    Parameters:
    - model_output (tuple): Выход модели, включающий токенов эмбеддинги и другие данные.
    - attention_mask (torch.Tensor): Маска внимания для указания значимости токенов.

    Returns:
    - torch.Tensor: Усредненный эмбеддинг.

    Examples:
    >>> embeddings = model_output[0]
    >>> mask = torch.tensor([[1, 1, 1, 0, 0]])
    >>> pooled_embedding = mean_pooling((embeddings,), mask)
    """
    # Получаем эмбеддинги токенов из выхода модели
    token_embeddings = model_output[0]

    # Расширяем маску внимания для умножения с эмбеддингами
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )

    # Умножаем каждый токен на его маску и суммируем
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)

    # Суммируем маски токенов и обрезаем значения, чтобы избежать деления на ноль
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    # Вычисляем усредненный эмбеддинг
    return sum_embeddings / sum_mask


def txt2embeddings(text: Union[str, List[str]], tokenizer, model, device: str = "cpu") -> torch.Tensor:
    """
    Преобразует текст в его векторное представление с использованием модели transformer.

    Parameters:
    - text (str): Текст для преобразования в векторное представление.
    - tokenizer: Токенизатор для предобработки текста.
    - model: Модель transformer для преобразования токенов в вектора.
    - device (str): Устройство для вычислений (cpu или cuda).

    Returns:
    - torch.Tensor: Векторное представление текста.

    Examples:
    >>> text = "Пример текста"
    >>> tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
    >>> model = AutoModel.from_pretrained("bert-base-multilingual-cased")
    >>> embeddings = txt2embeddings(text, tokenizer, model, device="cuda")
    """
    # Кодируем входной текст с помощью токенизатора
    if isinstance(text, str):
        text = [text]
    encoded_input = tokenizer(
        text,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=128,
    )
    # Перемещаем закодированный ввод на указанное устройство
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}

    # Получаем выход модели для закодированного ввода
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Преобразуем выход модели в векторное представление текста
    return mean_pooling(model_output, encoded_input["attention_mask"])


def load_models(model: str, device: str = "cpu", torch_dtype: str = "auto") -> tuple:
    """
    Загружает токенизатор и модель для указанной предобученной модели.

    Parameters:
    - model (str): Название предобученной модели, поддерживаемой библиотекой transformers.

    Returns:
    - tuple: Кортеж из токенизатора и модели.

    Examples:
    >>> tokenizer, model = load_models("ai-forever/sbert_large_nlu_ru")
    """
    # Загружаем токенизатор для модели
    tokenizer = AutoTokenizer.from_pretrained(model, device_map=device, torch_dtype=torch_dtype)
    
    # Загружаем модель
    model = AutoModel.from_pretrained(model, device_map=device, torch_dtype=torch_dtype)

    return tokenizer, model

### Описание класса Document

`Document` - модель для представления документа с контентом страницы и метаданными.


In [4]:
class Document(BaseModel):
    """
    Модель для представления документа с контентом страницы и метаданными.

    Attributes:
        page_content (str): Контент страницы документа.
        metadata (dict): Метаданные документа. По умолчанию пустой словарь.
        type (Literal["Document"]): Тип документа. По умолчанию "Document".

    Methods:
        to_dict: Возвращает словарь с контентом страницы и метаданными.
        lc_secrets: Свойство, возвращает пустой словарь для хранения секретных данных.
        lc_attributes: Свойство, возвращает пустой словарь для хранения атрибутов.
        try_neq_default: Статический метод, проверяет, отличается ли значение от значения по умолчанию для поля модели.

    Special Methods:
        __repr_args__: Метод для представления аргументов объекта в строке repr.

    """
    page_content: str
    metadata: dict = Field(default_factory=dict)
    type: Literal["Document"] = "Document"

    def to_dict(self) -> Dict[str, Any]:
        """
        Преобразует документ в словарь.

        Returns:
            dict: Словарь с контентом страницы и метаданными.
        """
        return {"page_content": self.page_content, **self.metadata}

    @property
    def lc_secrets(self) -> Dict[str, str]:
        """
        Возвращает пустой словарь для хранения секретных данных.

        Returns:
            dict: Пустой словарь.
        """
        return dict()

    @property
    def lc_attributes(self) -> Dict:
        """
        Возвращает пустой словарь для хранения атрибутов.

        Returns:
            dict: Пустой словарь.
        """
        return {}

    @classmethod
    def try_neq_default(cls, value: Any, key: str, model: BaseModel) -> bool:
        """
        Проверяет, отличается ли значение от значения по умолчанию для поля модели.

        Args:
            value (Any): Значение поля.
            key (str): Название поля.
            model (BaseModel): Модель.

        Returns:
            bool: Результат сравнения.
        """
        try:
            return model.model_fields[key].default != value
        except Exception:
            return True

    def __repr_args__(self) -> Any:
        """
        Представляет аргументы объекта в строке repr.

        Returns:
            Any: Аргументы объекта.
        """
        return [
            (k, v)
            for k, v in super().__repr_args__()
            if (k not in self.model_fields or self.try_neq_default(v, k, self))
        ]

### Tokenizer

`Tokenizer` — класс для токенизации текста.

### SentenceChunker

`SentenceChunker` — класс для разбиения текста на части по предложениям с использованием токенизатора.

### RecursiveChunker

`RecursiveChunker` — класс для разделения текста на части с учётом различных разделителей и ограничений на размер части.

In [5]:
@dataclass(frozen=True)
class Tokenizer:
    """
    Класс для токенизации текста.

    Attributes:
        chunk_overlap (int): Количество символов перекрытия между частями текста.
        tokens_per_chunk (int): Максимальное количество токенов в части.
        decode (Callable[[List[int]], str]): Функция декодирования списка токенов в строку.
        encode (Callable[[str], List[int]]): Функция кодирования строки в список токенов.
    """
    chunk_overlap: int
    tokens_per_chunk: int
    decode: Callable[[List[int]], str]
    encode: Callable[[str], List[int]]


class SentenceChunker:
    """
    Класс для разбиения текста на части по предложениям с использованием токенизатора.

    Attributes:
        chunk_overlap (int): Количество символов перекрытия между частями текста.
        model_name (str): Название модели для разбиения текста.
        tokens_per_chunk (Optional[int]): Максимальное количество токенов в части.
        add_start_index (bool): Флаг добавления индекса начала части.
        strip_whitespace (bool): Флаг удаления пробелов в начале и конце части.

    Methods:
        create_documents(texts: List[str], metadatas: Optional[List[dict]] = None) -> List[Document]:
            Создает документы на основе списка текстов.

        split_documents(documents: Iterable[Document]) -> List[Any]:
            Разбивает список документов на тексты и метаданные и вызывает метод create_documents.

        split_text(text: str) -> List[str]:
            Разбивает текст на части с помощью токенизатора.

        count_tokens(text: str) -> int:
            Считает количество токенов в тексте.
    """

    def __init__(
        self,
        chunk_overlap: int = 50,
        model_name: str = "ai-forever/sbert_large_nlu_ru",
        tokens_per_chunk: Optional[int] = None,
        device: str = "cpu",
        add_start_index: bool = False,
        strip_whitespace: bool = True,
        **kwargs: Any,
    ) -> None:

        try:
            from transformers import AutoTokenizer, AutoModel
        except ImportError:
            raise ImportError(
                "Пожалуйста, установите transformers с помощью `pip install transformers`."
            )
        self._chunk_overlap = chunk_overlap
        self.model_name = model_name
        self._device = device
        self._model = AutoModel.from_pretrained(self.model_name, device_map=device)
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name, device_map=device)
        self._initialize_chunk_configuration(tokens_per_chunk=tokens_per_chunk)
        self._add_start_index = add_start_index
        self._strip_whitespace = strip_whitespace

    def _initialize_chunk_configuration(
        self, *, tokens_per_chunk: Optional[int]
    ) -> None:
        """
        Инициализация конфигурации разбиения текста на части.

        Args:
            tokens_per_chunk (Optional[int]): Максимальное количество токенов в части.
        """

        # Получение максимального количества токенов модели.
        self.maximum_tokens_per_chunk = cast(
            int, self._model.config.max_position_embeddings
        )

        # Установка значения tokens_per_chunk в соответствии с переданным или максимальным значением.
        if tokens_per_chunk is None:
            self.tokens_per_chunk = self.maximum_tokens_per_chunk
        else:
            self.tokens_per_chunk = tokens_per_chunk

        # Проверка, что tokens_per_chunk не превышает максимальное значение токенов модели.
        if self.tokens_per_chunk > self.maximum_tokens_per_chunk:
            raise ValueError(
                f"Лимит токенов модели '{self.model_name}'"
                f" составляет: {self.maximum_tokens_per_chunk}."
                f" Аргумент tokens_per_chunk={self.tokens_per_chunk}"
                f" > максимального лимита токенов."
            )

    def create_documents(
        self, texts: List[str], metadatas: Optional[List[dict]] = None
    ) -> List[Document]:
        """
        Создание документов на основе текстов.

        Args:
            texts (List[str]): Список текстов для обработки.
            metadatas (Optional[List[dict]]): Список метаданных для каждого текста.

        Returns:
            List[Document]: Список документов.
        """

        # Создание пустых списков для текстов и метаданных.
        _metadatas = metadatas or [{}] * len(texts)
        documents = []

        # Итерация по текстам для создания документов.
        for i, text in enumerate(texts):
            index = 0
            previous_chunk_len = 0

            # Итерация по частям текста.
            for chunk in self.split_text(text):
                metadata = copy.deepcopy(_metadatas[i])

                # Добавление индекса начала части, если флаг установлен.
                if self._add_start_index:
                    offset = index + previous_chunk_len - self._chunk_overlap
                    index = text.find(chunk, max(0, offset))
                    metadata["start_index"] = index
                    previous_chunk_len = len(chunk)

                # Создание нового документа и добавление его в список документов.
                new_doc = Document(page_content=chunk, metadata=metadata)
                documents.append(new_doc)
        return documents

    def split_documents(self, documents: Iterable[Document]) -> List[Any]:
        """
        Разбивает список документов на отдельные тексты и метаданные,
        и вызывает метод create_documents для создания документов на основе текстов.

        Args:
            documents (Iterable[Document]): Итерируемый список документов.

        Returns:
            List[Any]: Список созданных документов.
        """
        # Извлечение текстов и метаданных из документов
        texts, metadatas = [], []
        for doc in documents:
            texts.append(doc.page_content)
            metadatas.append(doc.metadata)
        # Создание документов на основе текстов и метаданных
        return self.create_documents(texts, metadatas=metadatas)

    def split_text(self, text: str) -> List[str]:
        """
        Разбивает текст на части с помощью токенизатора.

        Args:
            text (str): Исходный текст.

        Returns:
            List[str]: Список частей текста.
        """

        def encode_strip_start_and_stop_token_ids(text: str) -> List[int]:
            # Функция для кодирования текста без токенов начала и конца
            return self._encode(text)[1:-1]

        # Создание экземпляра Tokenizer с необходимыми параметрами
        tokenizer = Tokenizer(
            chunk_overlap=self._chunk_overlap,
            tokens_per_chunk=self.tokens_per_chunk,
            decode=self.tokenizer.decode,
            encode=encode_strip_start_and_stop_token_ids,
        )
        # Разбиение текста на части на основе токенов
        return self.split_text_on_tokens(text=text, tokenizer=tokenizer)

    @staticmethod
    def split_text_on_tokens(*, text: str, tokenizer: Tokenizer) -> List[str]:
        """
        Разбивает текст на части с учетом токенизатора.

        Args:
            text (str): Исходный текст.
            tokenizer (Tokenizer): Токенизатор для разбиения текста.

        Returns:
            List[str]: Список частей текста.
        """
        # Разбиение текста на части на основе токенов
        splits: List[str] = []
        input_ids = tokenizer.encode(text)
        start_idx = 0
        cur_idx = min(start_idx + tokenizer.tokens_per_chunk, len(input_ids))
        chunk_ids = input_ids[start_idx:cur_idx]
        while start_idx < len(input_ids):
            splits.append(tokenizer.decode(chunk_ids))
            if cur_idx == len(input_ids):
                break
            start_idx += tokenizer.tokens_per_chunk - tokenizer.chunk_overlap
            cur_idx = min(start_idx + tokenizer.tokens_per_chunk, len(input_ids))
            chunk_ids = input_ids[start_idx:cur_idx]
        return splits

    def count_tokens(self, *, text: str) -> int:
        """
        Считает количество токенов в тексте.

        Args:
            text (str): Исходный текст.

        Returns:
            int: Количество токенов.
        """
        return len(self._encode(text))

    _max_length_equal_32_bit_integer: int = 2**32

    def _encode(self, text: str) -> List[int]:
        """
        Кодирует текст в токены с учетом максимальной длины.

        Args:
            text (str): Исходный текст.

        Returns:
            List[int]: Список токенов.
        """
        token_ids_with_start_and_end_token_ids = self.tokenizer.encode(
            text,
            max_length=self._max_length_equal_32_bit_integer,
            truncation="do_not_truncate",
        )
        return token_ids_with_start_and_end_token_ids


class RecursiveChunker:
    """
    Класс для разделения текста на части с учётом различных разделителей и ограничений на размер части.

    Args:
        separators (Optional[List[str]]): Список разделителей, по которым будет производиться разделение текста.
            По умолчанию содержит стандартные разделители: ["\n\n", "\n", " ", ""].
        keep_separator (bool): Флаг, определяющий, нужно ли сохранять разделители в итоговых частях текста. По умолчанию True.
        is_separator_regex (bool): Флаг, указывающий, являются ли разделители регулярными выражениями. По умолчанию False.
        length_function (Callable[[str], int]): Функция для определения длины текста. По умолчанию используется функция len.
        chunk_size (int): Максимальный размер части текста в символах. По умолчанию 256.
        chunk_overlap (int): Количество символов перекрытия между частями текста. По умолчанию 50.
        strip_whitespace (bool): Флаг, определяющий, нужно ли удалять пробелы в начале и конце частей текста. По умолчанию True.
        **kwargs: Дополнительные аргументы.

    Methods:
        _merge_splits(self, splits: Iterable[str], separator: str) -> List[str]:
            Объединяет части текста с разделителем separator, учитывая ограничения на размер части.

        _join_docs(self, docs: List[str], separator: str) -> Optional[str]:
            Объединяет список частей текста в один текст с разделителем separator.

        create_documents(self, texts: List[str], metadatas: Optional[List[dict]] = None) -> List[Document]:
            Создает список документов на основе текстов.

        split_documents(self, documents: Iterable[Document]) -> List[Any]:
            Разбивает документы на части и создает новые документы на основе частей текста.

        transform_documents(self, documents: Sequence[Document]) -> Sequence[Document]:
            Трансформирует документы, разбивая их на части и создавая новые документы на основе частей текста.

        _split_text_with_regex(text: str, separator: str, keep_separator: bool) -> List[str]:
            Разделяет текст на части с помощью регулярного выражения separator.

        _split_text(self, text: str, separators: List[str]) -> List[str]:
            Рекурсивно разбивает текст на части с учетом списка разделителей separators.

        split_text(self, text: str) -> List[str]:
            Разделяет текст на части с учетом установленных разделителей.

    """

    def __init__(
        self,
        separators: Optional[List[str]] = None,
        keep_separator: bool = True,
        is_separator_regex: bool = False,
        length_function: Callable[[str], int] = len,
        chunk_size: int = 256,
        chunk_overlap: int = 50,
        strip_whitespace: bool = True,
        add_start_index: bool = False,
        **kwargs: Any,
    ) -> None:
        self._separators = separators or ["\n\n", "\n", " ", ""]
        self._is_separator_regex = is_separator_regex
        self._chunk_size = chunk_size
        self._chunk_overlap = chunk_overlap
        self._length_function = length_function
        self._keep_separator = keep_separator
        self._add_start_index = add_start_index
        self._strip_whitespace = strip_whitespace

    def _merge_splits(self, splits: Iterable[str], separator: str) -> List[str]:
        """
        Объединяет части текста с разделителем separator, учитывая ограничения на размер части.

        Args:
            splits (Iterable[str]): Список частей текста.
            separator (str): Разделитель для объединения частей.

        Returns:
            List[str]: Список объединенных частей текста.
        """
        separator_len = self._length_function(separator)

        docs = []
        current_doc: List[str] = []
        total = 0
        for split in splits:
            _len = self._length_function(split)
            if (
                total + _len + (separator_len if len(current_doc) > 0 else 0)
                > self._chunk_size
            ):
                if total > self._chunk_size:
                    logger.warning(
                        f"Создан фрагмент размером {total}, "
                        f"который длиннее указанного {self._chunk_size}"
                    )

                if len(current_doc) > 0:
                    # Объединяем текущий документ и добавляем его в список объединенных документов.
                    doc = self._join_docs(current_doc, separator)
                    if doc is not None:
                        docs.append(doc)

                    # Удаляем из текущего документа части, которые не поместились в текущий чанк.
                    while total > self._chunk_overlap or (
                        total + _len + (separator_len if len(current_doc) > 0 else 0)
                        > self._chunk_size
                        and total > 0
                    ):
                        total -= self._length_function(current_doc[0]) + (
                            separator_len if len(current_doc) > 1 else 0
                        )
                        current_doc = current_doc[1:]
            current_doc.append(split)
            total += _len + (separator_len if len(current_doc) > 1 else 0)

        # Объединяем оставшиеся части текста в последний документ.
        doc = self._join_docs(current_doc, separator)
        if doc is not None:
            docs.append(doc)
        return docs

    def _join_docs(self, docs: List[str], separator: str) -> Optional[str]:
        """
        Объединяет список частей текста в один текст с разделителем separator.

        Args:
            docs (List[str]): Список частей текста.
            separator (str): Разделитель для объединения частей.

        Returns:
            Optional[str]: Объединенный текст или None, если текст пуст.
        """
        text = separator.join(docs)
        if self._strip_whitespace:
            text = text.strip()
        if text == "":
            return None
        else:
            return text

    def create_documents(
        self, texts: List[str], metadatas: Optional[List[dict]] = None
    ) -> List[Document]:
        """
        Создает список документов на основе текстов.

        Args:
            texts (List[str]): Список текстов для обработки.
            metadatas (Optional[List[dict]]): Список метаданных для каждого текста.

        Returns:
            List[Document]: Список созданных документов.
        """
        # Инициализация метаданных для текстов, если они не заданы.
        _metadatas = metadatas or [{}] * len(texts)
        documents = []

        for i, text in enumerate(texts):
            index = 0
            previous_chunk_len = 0

            for chunk in self.split_text(text):
                metadata = copy.deepcopy(_metadatas[i])

                if self._add_start_index:
                    offset = index + previous_chunk_len - self._chunk_overlap
                    # Находим индекс начала текущей части в исходном тексте.
                    index = text.find(chunk, max(0, offset))
                    metadata["start_index"] = index
                    previous_chunk_len = len(chunk)

                # Создаем новый документ на основе части текста и метаданных.
                new_doc = Document(page_content=chunk, metadata=metadata)
                documents.append(new_doc)
        return documents

    def split_documents(self, documents: Iterable[Document]) -> List[Any]:
        """
        Разбивает документы на части и создает новые документы на основе частей текста.

        Args:
            documents (Iterable[Document]): Список документов для разбиения.

        Returns:
            List[Any]: Список созданных документов или их частей.
        """
        texts, metadatas = [], []
        for doc in documents:
            texts.append(doc.page_content)
            metadatas.append(doc.metadata)
        return self.create_documents(texts, metadatas=metadatas)

    @staticmethod
    def _split_text_with_regex(
        text: str, separator: str, keep_separator: bool
    ) -> List[str]:
        """
        Разделяет текст на части с использованием регулярного выражения separator.

        Args:
            text (str): Исходный текст.
            separator (str): Регулярное выражение для разделения текста.
            keep_separator (bool): Флаг, указывающий, нужно ли сохранять разделители.

        Returns:
            List[str]: Список частей текста.
        """
        if separator:
            if keep_separator:
                splits = re.split(f"({separator})", text)
                # Объединяем разделители с соответствующими частями текста.
                splits = [splits[i] + splits[i + 1] for i in range(1, len(splits), 2)]
                # Если количество разделителей нечетное, добавляем последний элемент.
                if len(splits) % 2 == 0:
                    splits += splits[-1:]
                # Добавляем первоначальный элемент текста в начало списка.
                splits = [splits[0]] + splits
            else:
                splits = re.split(separator, text)
        else:
            # Если разделитель не указан, разбиваем текст на символы.
            splits = list(text)
        # Удаляем пустые строки из списка.
        return [s for s in splits if s != ""]

    def transform_documents(self, documents: Sequence[Document]) -> Sequence[Document]:
        return self.split_documents(list(documents))

    def _split_text(self, text: str, separators: List[str]) -> List[str]:
        """
        Рекурсивно разбивает текст на части с учетом списка разделителей separators.

        Args:
            text (str): Исходный текст.
            separators (List[str]): Список разделителей для разбиения текста.

        Returns:
            List[str]: Список частей текста.
        """
        # Инициализация пустого списка для хранения окончательных частей текста.
        final_chunks = []

        # Инициализация разделителя как последнего в списке.
        current_separator = separators[-1]

        # Инициализация списка разделителей.
        remaining_separators = []

        # Проверка каждого разделителя в списке.
        for i, separator in enumerate(separators):
            regex_separator = (
                separator if self._is_separator_regex else re.escape(separator)
            )

            # Если разделитель пустой, устанавливаем его и прерываем цикл.
            if separator == "":
                current_separator = separator
                break

            # Если найден разделитель в тексте, устанавливаем его и запоминаем оставшиеся разделители.
            if re.search(regex_separator, text):
                current_separator = separator
                remaining_separators = separators[i + 1 :]
                break

        # Применяем регулярное выражение к разделителю.
        regex_separator = (
            current_separator
            if self._is_separator_regex
            else re.escape(current_separator)
        )

        # Разделяем текст на части с помощью регулярного выражения.
        splits = self._split_text_with_regex(
            text, regex_separator, self._keep_separator
        )

        # Создаем список для хранения "хороших" частей текста.
        good_splits = []

        # Инициализация разделителя как пустой строки или исходного разделителя.
        current_separator = "" if self._keep_separator else current_separator

        # Проходим по каждой части текста после разделения.
        for split in splits:

            # Если длина части меньше максимального размера части, добавляем её в "хорошие" части.
            if self._length_function(split) < self._chunk_size:
                good_splits.append(split)
            else:

                # Если часть больше максимального размера, объединяем "хорошие" части и добавляем в итоговый список.
                if good_splits:
                    merged_text = self._merge_splits(good_splits, current_separator)
                    final_chunks.extend(merged_text)
                    good_splits = []

                # Если есть новые разделители, разбиваем текущую часть на еще более мелкие части.
                if not remaining_separators:
                    final_chunks.append(split)
                else:
                    other_chunks = self._split_text(split, remaining_separators)
                    final_chunks.extend(other_chunks)

        # Добавляем "хорошие" части, если они остались после обработки.
        if good_splits:
            merged_text = self._merge_splits(good_splits, current_separator)
            final_chunks.extend(merged_text)

        # Возвращаем итоговый список частей текста.
        return final_chunks

    def split_text(self, text: str) -> List[str]:
        """
        Разделяет текст на части с учетом установленных разделителей.

        Args:
            text (str): Исходный текст.

        Returns:
            List[str]: Список частей текста.
        """
        return self._split_text(text, self._separators)

### Константы и настройки

- `TABLE_NAME`: Название таблицы в базе данных ClickHouse, куда будут добавляться данные.
- `MODEL_EMB_NAME`: Название модели для получения эмбеддингов текста.
- `HOST`: Хост, на котором запущен сервер ClickHouse.
- `PORT`: Порт сервера ClickHouse.
- `DEVICE`: Устройство для вычислений PyTorch (CPU или CUDA).

In [6]:
TABLE_NAME = "table10"
MODEL_EMB_NAME = "ai-forever/sbert_large_nlu_ru"
HOST = "f666-81-5-106-50.ngrok-free.app"
PORT = "80"
DEVICE = "cuda"

### Данные для добавления в базу данных

- `text`: Текст документа.
- `title`: Заголовок документа.
- `url`: URL источника документа.
- `date`: Дата документа.
- `number`: Номер документа.
- `file_type`: Тип файла.
- `data`: Список словарей с данными для добавления.

In [7]:
text = """Организаторам торговли Профессиональным участникам рынка ценных бумаг , Москва, ул. Неглинная, д. 12, к. В. тел. 499 300-30-00, 8 800 300-30-00 Операторам информационных систем, в которых осуществляется выпуск цифровых финансовых активов От На от Информационное письмо Банка России об отдельных видах финансовых инструментов и цифровых финансовых активов Операторам обмена цифровых финансовых активов В целях защиты прав и интересов инвесторов и поддержания доверия к финансовому рынку Российской Федерации Банк России рекомендует следующее. 1. Организаторам торговли на основании пункта 6 статьи 14 Федерального закона от 22.04.1996 39-ФЗ О рынке ценных бумаг рекомендуется отказывать в допуске к организованным торгам ценных бумаг российских и иностранных эмитентов обязанных по ценным бумагам лиц, права владельцев на получение выплат по которым и или размер выплат по которым размер дохода либо доходность которых зависят1 от изменения одного или нескольких показателей, указанных в пункте 5 настоящего Информационного письма Банка России, а также ценных бумаг, в соответствии с условиями выпуска которых эмитент обязанное по ценным бумагам лицо осуществляет деятельность по оказанию услуг, направленных на обеспечение выпуска цифровой валюты и или совершения гражданско 1 В соответствии с условиями их выпуска, содержащимися в решении о выпуске указанных ценных бумаг или в документе, составленном в соответствии с личным законом иностранного эмитента обязанного по ценным бумагам лица и содержащем данные, достаточные для установления объема прав, закрепленных ценными бумагами иностранного эмитента обязанного по ценным бумагам лица. 2 правовых сделок и или операций, влекущих за собой переход цифровой валюты от одного обладателя к другому далее ценные бумаги организатора обращения цифровой валюты. Также организаторам торговли рекомендуется воздерживаться от оказания услуг по проведению организованных торгов, на которых заключаются договоры, предусматривающие одну или несколько из обязанностей, указанных в подпункте 5.6 пункта 5 настоящего Информационного письма Банка России. 2. Профессиональным участникам рынка ценных бумаг рекомендуется воздерживаться от предложения клиентам физическим лицам, не являющимся квалифицированными инвесторами: 2.1. Приобретать ценные бумаги, права владельцев на получение выплат по которым и или размер выплат по которым размер дохода либо доходность которых зависят2 от изменения одного или нескольких показателей, указанных в пункте 5 настоящего Информационного письма Банка России. 2.2. Приобретать ценные бумаги организатора обращения цифровой валюты. 2.3. Заключать договоры доверительного управления, допускающие приобретение ценных бумаг, права владельцев на получение выплат по которым и или размер выплат по которым размер дохода либо доходность которых зависят3 от изменения одного или нескольких показателей, указанных в пункте 5 настоящего Информационного письма Банка России, и вносить в действующие договоры доверительного управления соответствующие изменения. 2.4. Заключать договоры доверительного управления, допускающие приобретение ценных бумаг организатора обращения цифровой валюты, 2 См. сноску 1. 3 См. сноску 1. 3 и вносить в действующие договоры доверительного управления соответствующие изменения. 2.5. Заключать договоры, указанные в подпункте 5.6 пункта 5 настоящего Информационного письма Банка России. 3. Операторам информационных систем, в которых осуществляется выпуск цифровых финансовых активов далее ЦФА, рекомендуется не оказывать услуги по организации выпуска, учета и обращения ЦФА, право на получение выплат по которым и или размер выплат по которым размер дохода либо доходность которых зависят4 от изменения одного или нескольких показателей, указанных в пункте 5 настоящего Информационного письма Банка России. 4. Операторам обмена ЦФА рекомендуется не оказывать услуги по организации совершения сделок с ЦФА в том числе выпущенных в информационных системах, организованных в соответствии с иностранным правом, право на получение выплат по которым и или размер выплат по которым размер дохода либо доходность которых зависят5 от изменения одного или нескольких показателей, указанных в пункте 5 настоящего Информационного письма Банка России. 5. В целях применения настоящего Информационного письма Банка России организациям, указанным в пунктах 1 4 настоящего Информационного письма Банка России, рекомендуется руководствоваться следующим перечнем показателей: 5.1. Цена курс цифровых валют. 5.2. Цена на имущественные права, которые не признаются в соответствии с законодательством Российской Федерации цифровыми правами или ценными бумагами и которые включают: денежные требования; возможность осуществления прав по эмиссионным ценным бумагам; права участия в капитале непубличного акционерного общества; право требовать 4 В соответствии с условиями, содержащимися в решении о выпуске ЦФА. 5 См. сноску 4. 4 передачи эмиссионных ценных бумаг; право требовать передачи вещи вещей; право требовать передачи исключительных прав на результаты интеллектуальной деятельности, прав использования результатов интеллектуальной деятельности; право требовать выполнения работ; право требовать оказания услуг, осуществление, распоряжение или ограничение распоряжения которыми возможны в информационной системе без обращения к третьему лицу. 5.3. Цена на ценные бумаги организатора обращения цифровой валюты. 5.4. Цена на ценные бумаги, права владельцев на получение выплат по которым и или размер выплат по которым размер дохода либо доходность которых зависят от изменения показателей, указанных в подпунктах 5.1 5.3, 5.5 и 5.6 настоящего пункта. 5.5. Значение индекс, рассчитываемое рассчитываемый на основании показателей, указанных в подпунктах 5.1 5.4, 5.6 настоящего пункта. 5.6. Цена имущественных прав из договоров, в том числе являющихся иностранными финансовыми инструментами, предусматривающими одну или несколько из следующих обязанностей: обязанность сторон или стороны договора периодически или единовременно уплачивать денежные суммы, в том числе в случае предъявления требований другой стороной, в зависимости от изменения показателей, указанных в подпунктах 5.1 5.5 настоящего пункта; обязанность стороны или сторон договора передать другой стороне цифровые валюты, ценные бумаги, имущественные права, указанные в подпунктах 5.1 5.4 настоящего пункта, или заключить договор о приобретении цифровых валют, ценных бумаг, имущественных прав, указанных в подпунктах 5.1 5.4 настоящего пункта; обязанность стороны или сторон договора на условиях, определенных при заключении договора, в случае предъявления требования другой стороной купить или продать цифровые валюты, ценные бумаги, имущественные права, указанные в подпунктах 5.1 5.4 настоящего пункта; 5 обязанность стороны или сторон договора заключить договор, предусматривающий одну или несколько из обязанностей, указанных в абзацах втором, третьем или четвертом настоящего подпункта. С даты опубликования настоящего Информационного письма Банка России Информационное письмо Банка России об отдельных видах финансовых инструментов от 19.07.2021 ИН-06-59 52 отменяется. Настоящее Информационное письмо Банка России подлежит опубликованию в Вестнике Банка России и размещению на официальном сайте Банка России в информационно-телекоммуникационной сети Интернет. Заместитель Председателя Банка России Ф.Г. Габуния"""  # String
title = "Об отмене Методических рекомендаций Банка России по усилению контроля операторами по переводу денежных средств за деятельностью банковских платежных агентов от 14.04.2016 11-МР"  # String
url = "https://cbr.ru/Crosscut/LawActs/File/7673"  # String
date = "от 11.01.2024"  # String
number = "04-45/107"  # String
file_type = "from_test"
data = [{
    "text": text,
    "number": number,
    "date": date,
    "title": title,
    "file_type": file_type,
    "url": url,
}]

### Загрузка модели и токенизатора

Загружаем модель и токенизатор для создания векторных представлений текста.

In [8]:
tokenizer, model = load_models(MODEL_EMB_NAME, device=DEVICE)

### Инициализация разделителей

Создаем объекты для разделения текста на части.

- `RecursiveChunker`: Разделитель текста на части с учетом различных разделителей и ограничений на размер части.
    - `chunk_overlap=20`: Количество символов перекрытия между частями текста.
    - `chunk_size=256`: Максимальный размер части текста в символах.

- `SentenceChunker`: Разделитель текста на части по предложениям с использованием токенизатора.
    - `model_name=MODEL_EMB_NAME`: Название модели для разбиения текста.
    - `device="cpu"`: Устройство для выполнения вычислений (CPU).

In [9]:
recursive_splitter = RecursiveChunker(chunk_overlap=20, chunk_size=256)
sentence_splitter = SentenceChunker(model_name=MODEL_EMB_NAME, device="cpu")

### Подготовка данных для разделения

Для каждого элемента данных:
1. Извлекаем текст и сохраняем его в отдельный список `text`.
2. Создаем копии элемента данных с разными типами разделения:
    - Один для рекурсивного разделения с метаданными `metadata_recursive`.
    - Другой для разделения по предложениям с метаданными `metadata_sentence`.


In [10]:
text = []
metadata_recursive = []
metadata_sentence = []
for item in data:
    text.append(item.pop("text"))
    item2 = item.copy()
    item["chunk_type"] = "Recursive"
    item2["chunk_type"] = "Sentence"
    metadata_recursive.append(item)
    metadata_sentence.append(item2)

### Разбиение текста на части

1. Создаем документы и разбиваем текст на части при помощи `RecursiveChunker`.
2. Оцениваем время выполнения разбиения.

3. Создаем документы и разбиваем текст на части при помощи `SentenceChunker`.
4. Оцениваем время выполнения разбиения.

5. Соединяем результаты обоих методов разбиения.
6. Выводим общее количество документов и частей, а также общее время выполнения.


In [11]:
start = time.time()

print("Начинаем разбиение текста")
docs_recursive = recursive_splitter.create_documents(text, metadata_recursive)
chunking_recursive = recursive_splitter.split_documents(docs_recursive)

print(f"Разбиение {len(docs_recursive)} документов при помощи RecursiveChunker на {len(chunking_recursive)} частей заняло {time.time() - start} секунд")

start_sentence = time.time()
docs_sentence = sentence_splitter.create_documents(text, metadata_sentence)
chunking_sentence = sentence_splitter.split_documents(docs_sentence)

print(f"Разбиение {len(docs_sentence)} документов при помощи SentenceChunker на {len(chunking_sentence)} частей заняло {time.time() - start_sentence} секунд")

all_documents = docs_recursive + docs_sentence
all_chunks = chunking_recursive + chunking_sentence
print(f"Общее количество документов: {len(all_documents)}\nОбщее количество частей: {len(all_chunks)}\nОбщее время: {time.time() - start}")
print(*all_documents, sep="\n")
print()
print(*all_chunks, sep="\n")


Начинаем разбиение текста
Разбиение 32 документов при помощи RecursiveChunker на 48 частей заняло 0.0024454593658447266 секунд
Разбиение 3 документов при помощи SentenceChunker на 3 частей заняло 0.02603626251220703 секунд
Общее количество документов: 35
Общее количество частей: 51
Общее время: 0.028705835342407227
page_content='торговли торговли Профессиональным участникам рынка ценных бумаг , Москва, ул. Неглинная, д. 12, к. В. тел. 499 300-30-00, 8 800 300-30-00 Операторам информационных систем, в которых осуществляется выпуск цифровых финансовых активов От На от' metadata={'number': '04-45/107', 'date': 'от 11.01.2024', 'title': 'Об отмене Методических рекомендаций Банка России по усилению контроля операторами по переводу денежных средств за деятельностью банковских платежных агентов от 14.04.2016 11-МР', 'file_type': 'from_test', 'url': 'https://cbr.ru/Crosscut/LawActs/File/7673', 'chunk_type': 'Recursive'}
page_content='активов От На от Информационное письмо Банка России об отдел

### Получение текстовых данных

1. Извлекаем текстовые данные из всех документов.

In [12]:
text_data = [item.to_dict()["page_content"] for item in all_chunks]

### Получение векторных представлений

1. Преобразуем текстовые данные в векторные представления при помощи модели и токенизатора.


In [13]:
embeddings = txt2embeddings(text_data, tokenizer, model, device=DEVICE)

### Подключение к ClickHouse

3. Устанавливаем соединение с ClickHouse и проверяем доступность сервера.

**<font color='red'>Внимание!</font>** Если сервер недоступен, свяжитесь с [хостом](https://t.me/allelleo).

In [14]:
client = clickhouse_connect.get_client(host=HOST, port=PORT)
print("Ping:", client.ping())

Ping: True


### Отправка запросов
Для каждого элемента в `all_chunks`:
1. Преобразуем элемент в словарь.
2. Собираем `embeddings` в строку.
3. Формируем строку значений для запроса в базу данных.
4. Создаем SQL-запрос для вставки данных в таблицу.
5. Выполняем запрос к ClickHouse.

In [15]:

for i, item in enumerate(all_chunks):
    item = item.to_dict()

    vectors = ",".join([str(float(vector)) for vector in embeddings[i]])
    values = f"('{text}','{number}','{date}','{title}','{url}',,'({vectors})'),"
    query =  f"""INSERT INTO "{TABLE_NAME}"("Text","Number","Date","Title","FileType","Url","ChunkType", "Embedding") VALUES('{item.get('page_content')}', '{item.get('number')}', '{item.get('date')}', '{item.get('title')}', '{item.get('file_type')}', '{item.get('url')}', '{item.get('chunk_type')}', ({vectors}))"""
    client.command(query)